In [ ]:
!nvidia-smi

In [ ]:
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/einops-0.6.1-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/model_index-0.1.11-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/modelindex-0.0.2-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/ordered_set-4.1.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mat4py-0.5.0-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torch-1.12.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torchvision-0.13.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmcv-2.0.1-cp310-cp310-manylinux1_x86_64.whl 
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/openmim-0.3.9-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmengine-0.8.1-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/addict-2.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmdet-3.1.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/terminaltables-3.1.10-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmpretrain-1.0.0-py2.py3-none-any.whl

In [ ]:
!cp -r /kaggle/input/ensemble-boxes-104 ./
!pip install ./ensemble-boxes-104/ensemble_boxes-1.0.4/ 

### Importing Libraries

In [ ]:
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools.coco import COCO
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import os
import cv2, glob
import matplotlib.pyplot as plt
import wandb
from PIL import Image
import gc
sample = None
import mmcv
from glob import glob
import matplotlib.pyplot as plt
from mmdet.apis import init_detector, inference_detector
from ensemble_boxes import *
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
import mmdet as mmdet
print(mmdet.__version__)
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib
from mmdet.apis import init_detector, inference_detector
from mmengine.config import Config

### Function

In [ ]:
def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str

def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

def combine_masks(masks, mask_threshold):
    """
    combine masks into one image
    """
    maskimg = np.zeros((HEIGHT, WIDTH))
    # print(len(masks.shape), masks.shape)
    for m, mask in enumerate(masks,1):
        maskimg[mask>mask_threshold] = m
    return maskimg

### Test Dataset and DataLoader

In [ ]:
all_imgs = glob('D:/CodeProject/HuBMAP2023/Data/test/*.tif')

df = pd.read_csv('/kaggle/input/hubmap-hacking-the-human-vasculature/tile_meta.csv')
valid_ids = df.query('dataset == 1')['id']  
all_imgs = sorted(list('/kaggle/input/hubmap-hacking-the-human-vasculature/train/'+valid_ids+'.tif'))

### Model

In [ ]:
config_file = 'D:/CodeProject/HuBMAP2023/SwinTransformer-config.py'
checkpoint_file = 'D:/CodeProject/HuBMAP2023/Data/work_dirs/SwinTransformer-config/best_coco_segm_mAP_epoch_4.pth'

cfg = Config.fromfile(config_file )
SwinTransformer = init_detector(cfg, checkpoint_file, device=device)

In [ ]:
config_file = 'D:/CodeProject/HuBMAP2023/Convext-xlarge-config.py'
checkpoint_file = 'D:/CodeProject/HuBMAP2023/Data/work_dirs/Convext-xlarge-config/best_coco_segm_mAP_epoch_4.pth'

cfg = Config.fromfile(config_file )

Convext_v1_l = init_detector(cfg, checkpoint_file, device=device)

In [ ]:
config_file = 'D:/CodeProject/HuBMAP2023/EfficientNet-b8-config.py'
checkpoint_file = 'D:/CodeProject/HuBMAP2023/Data/work_dirs/EfficientNet-b8-config/best_coco_segm_mAP_epoch_4.pth'

cfg = Config.fromfile(config_file )
EfficientNet_b8 = init_detector(cfg, checkpoint_file, device=device)

In [ ]:
config = {
    'effb8':{#0.523
        'scale':[(1024, 1024),(960, 960)],
        'mask_thr_binary':0.5,
        'weight':0.3, 
        'model':EfficientNet_b8,
    },
    'convext_xl_1':{#0.5523
        'scale':[(1024, 1024),(960, 960)],
        'mask_thr_binary':0.56,
        'weight':0.6,   
        'model':Convext_v1_l,
    },
    'swin_3':{#0.5866
        'scale':[(1440, 1440),(1024, 1024),(1280, 1280),],
        'mask_thr_binary':0.56,
        'weight':1.0,   
        'model':SwinTransformer,
    },
}

### Predicting

In [ ]:
mask_threshold_dict = {1: 0.3, 0: 0.99, 2:  0.80}
min_score_dict = {1: 0.01, 0: 0.99, 2: 0.80}

In [ ]:
import random
def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

In [ ]:
def draw_bboxes(img_draw, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'voc_pascal', class_name = False, line_thickness = 2):

    image = img_draw.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors

    if bbox_format == 'yolo':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:

                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
    elif bbox_format == 'voc_pascal':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

In [ ]:
def show_img(img_draw, bboxes, bbox_format='voc_pascal', bbox_colors = None):
    names  = ['blood']*len(bboxes)
    labels = [0]*len(bboxes)
    img_draw    = draw_bboxes(img_draw = img_draw,
                           bboxes = bboxes,
                           classes = names,
                           class_ids = labels,
                           class_name = True,
                           colors = colors if bbox_colors is None else bbox_colors,
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img_draw).resize((512, 512))

In [ ]:
def nms_predictions(classes, scores, bboxes, masks, 
                    iou_th=.7, shape=(512, 512)):
    he, wd = shape[0], shape[1]
    boxes_list = [[[x[0] / wd, x[1] / he, x[2] / wd, x[3] / he] for x in bboxes]]
    scores_list = [[x for x in scores]]
    classes_list = [[x for x in classes]]
    nms_bboxes, nms_scores, nms_classes = non_maximum_weighted(
        boxes_list, 
        scores_list, 
        classes_list, 
        weights=None,
        iou_thr=iou_th,skip_box_thr=0.01
    )
    nms_masks = []
    for s in nms_scores:
        nms_masks.append(masks[scores.index(s)])
    #nms_scores, nms_classes, nms_masks, nms_bboxes = zip(*sorted(zip(nms_scores, nms_classes, nms_masks, nms_bboxes), reverse=True))
    return nms_classes, nms_scores, nms_masks, nms_bboxes

In [ ]:
ids = []
heights = []
widths = []
prediction_strings = []
outkeys = ["bboxes", "masks", "labels", "scores"]
#predition = defaultdict()
for img in tqdm(all_imgs):
    img_array = mmcv.imread(img, channel_order='rgb')
    [h, w, c] = img_array.shape
    img_mask = []
    img_bboxes = []
    img_scores = []
    img_labels = []
    for d in config:
        model = config[d]['model']
        model.cfg.model.test_cfg.rcnn.mask_thr_binary = config[d]['mask_thr_binary']
        model.cfg.model.test_cfg.rcnn.score_thr = 0.01
        masks_list = []
        bboxes_list = []
        scores_list = []
        labels_list = []
        for scale in config[d]['scale']:
            model.cfg.test_dataloader.dataset.pipeline[1]['scale'] = scale
            pred = inference_detector(model, img_array)
            pred_dict = {k:pred.pred_instances[k] for k in outkeys}
            masks_list.append(pred_dict['masks'])
            bboxes_list.append(pred_dict['bboxes'])
            scores_list.append(pred_dict['scores']*config[d]['weight'])

            labels_list.append(pred_dict['labels'])

        masks = torch.cat(masks_list).cpu().numpy()
        bboxes = torch.cat(bboxes_list).cpu().numpy()
        scores = torch.cat(scores_list).cpu().numpy()
        labels = torch.cat(labels_list).cpu().numpy()

        masks = masks[labels==1]
        bboxes = bboxes[labels==1]
        scores = scores[labels==1]
        labels = labels[labels==1]

        nms_classes, nms_scores, nms_masks, nms_bboxes = nms_predictions(labels, list(scores), bboxes, masks, iou_th=0.58)
        img_mask.append(nms_masks)
        img_bboxes.append(nms_bboxes)
        img_scores.append(nms_scores)
        img_labels.append(nms_classes)
    masks = np.concatenate(img_mask)
    bboxes = np.concatenate(img_bboxes)
    scores = np.concatenate(img_scores)
    labels = np.concatenate(img_labels)
    nms_classes, nms_scores, nms_masks, nms_bboxes = nms_predictions(labels, list(scores), bboxes, masks, iou_th=0.72)
    pred_string=""
    m = 0
    for i, mask in enumerate(nms_masks):
        # Filter-out low-scoring results.
        score = nms_scores[i]
        label = nms_classes[i]

        binary_mask = mask > 0.5

        #dilation
        binary_mask = binary_mask.astype(np.uint8)
        kernel = np.ones(shape=(3, 3), dtype=np.uint8)
        binary_mask = cv2.dilate(binary_mask, kernel, iterations=1)

        binary_mask = binary_mask.astype(bool)


        encoded = encode_binary_mask(binary_mask)
        #if label != 1: continue
        if m == 0:
            pred_string += f"0 {score} {encoded.decode('utf-8')}"
            m = m+1
        else:
            pred_string += f" 0 {score} {encoded.decode('utf-8')}"
        #print(pred_classes[i])

    ids.append(str(img).split('.')[0].split('/')[-1])
    heights.append(h)
    widths.append(w)
    prediction_strings.append(pred_string)

submission = pd.DataFrame()
submission['id'] = ids
submission['height'] = heights
submission['width'] = widths
submission['prediction_string'] = prediction_strings
submission = submission.set_index('id')
submission.to_csv("submission.csv")
submission.head()